In [1]:

import matplotlib.pyplot as plt

import seaborn as sns
import plotly.express as px
from scipy.stats import spearmanr
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Paleta corporativa
CORP_PALETTE =  [ "#1c8074","#666666", "#94af92", "#666666", "#f9ee77", "#f5ad68", "#c76931"]


In [2]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, create_subplot_grid, plot_bar, plot_correlation_grid, plot_scatter
from core.s3 import S3AssetManager

In [3]:
notebook_name = "comayma_micotoxinas"
s3 = S3AssetManager(notebook_name=notebook_name)

In [12]:
base = "raw/comayma/"
df_toxinas_24 = s3.read_excel(f"{base}Micotoxinas y Microbiología 2024.xlsx", sheet_name="Micotoxina PT", skiprows=7)
df_toxinas_25 = s3.read_excel(f"{base}Micotoxinas y Microbiología 2025 (2) (1).xlsx", sheet_name="Micotoxina PT", skiprows=7)
df_toxinas_25_Sep = s3.read_excel(f"{base}Micotoxinas y Microbiología 2025 (2) (2).xlsx", sheet_name="Micotoxina PT", skiprows=7)
toxinas = pd.concat([df_toxinas_24, df_toxinas_25, df_toxinas_25_Sep])




col = 'TEMP/HUM.\nAMBIENTE'
parts = (toxinas[col].astype(str).str.strip()
         .str.replace(r"%", "", regex=True)
         .str.split("/", n=1, expand=True))

toxinas["temp"]      = pd.to_numeric(parts[0].str.replace(",", ".", regex=False), errors="coerce")
toxinas["pct_hum"]  = pd.to_numeric(parts[1].str.replace(",", ".", regex=False), errors="coerce")  # 44, 47.9, etc.
toxinas["temp"]      = pd.to_numeric(parts[0].str.replace(",", ".", regex=False), errors="coerce")
toxinas[['TEMP/HUM.\nAMBIENTE' , "temp",'pct_hum']]

toxinas = s3.read_csv(f"{base}toxinas_base.csv")

In [13]:
toxinas["ESPECIE"].value_counts()

CERDOS               118
POSTURA PONEDORAS    108
FIN BROILER           60
GANADO LECH.          29
REPRODUCTORA          23
POSTURA PONEDORA      20
LEVANTE PONEDORAS     10
GANADO ENG.            7
LEVANTE BROILER        6
GANADO                 6
REPRODUCOTRA           5
CAPRINO                4
CONEJO                 2
POSTURA                2
GANADO TERN.           2
Name: ESPECIE, dtype: int64

In [14]:
cols_toxinas  = ['OP-Lote',"AFLA-PPB",'OCRA-PPB','T2-PPB','ZEARA-PPB','FUMO-PPM', 'DON-PPM','temp', 'pct_hum']
for col in cols_toxinas:
    toxinas[col] = pd.to_numeric(toxinas[col], errors='coerce')

In [15]:
toxinas["Fecha Producción"] = pd.to_datetime(
    toxinas["Fecha Producción"]
)
toxinas["anno_mes"] = pd.to_datetime(
    toxinas["Fecha Producción"], errors="coerce"
).dt.strftime("%Y-%m")


In [16]:
prod_last_month = toxinas[toxinas["Fecha Producción"]>='2025-07-01']['Producto'].unique()
prod_last_month

array(['FINALIZADOR BROILER', 'PONED. W80 FASE 2 MVA 100-105',
       'FINALIZADOR POLLO HARINA NTC ASO PLUS', 'GESTIVAC HARINA ASO',
       'LACTOMAYMA HARINA 22', 'LACTOMAYMA HARINA 18',
       'PONED. W80 IMP. MVA 75-80', 'PONED. W80 FASE 1 MVA 100-105',
       'NUTRIMAYMA FIN BROILER HA', 'LACTOMAYMA 18 HARINA ASO',
       'PONED. W80 IMP. MVA 80-85', 'FORRAMAYMA GANADO HARINA',
       'PONED. W80 IMP. MVA 90-95'], dtype=object)

In [17]:
#[toxinas["Fecha Producción"]>='2025-07-01']
#toxinas[(toxinas['Producto'].isin(prod_last_month)) & (toxinas["Fecha Producción"]>='2025-01-01')]
toxinas_mes_producto = toxinas[toxinas["Fecha Producción"]>='2025-01-01'].groupby(["Fecha Producción"]).agg(
    AFLATOXINA = ('AFLA-PPB', 'mean'),#
    #ocra = ('OCRA-PPB', 'mean'),
    #t2 = ('T2-PPB', 'mean'),
    ZEARALENONA = ('ZEARA-PPB', 'mean'),#
    FUMONISINA = ('FUMO-PPM', 'mean'),#
    DON = ('DON-PPM', 'mean'),#
    temp = ('temp', 'mean'),
    pct_hum = ('pct_hum', 'mean')

).reset_index()
toxinas_mes_producto

,Fecha Producción,AFLATOXINA,ZEARALENONA,FUMONISINA,DON,temp,pct_hum
0,2025-01-03,3.91,56.39,0.39,0.58,21.1,48.0
1,2025-01-06,NaN,69.47,1.15,0.20,21.1,48.0
2,2025-01-08,NaN,19.76,1.71,0.18,22.9,50.0
3,2025-01-10,0.58,11.60,NaN,1.10,22.9,50.0
4,2025-01-12,0.70,12.00,1.90,0.87,23.5,47.0
...,...,...,...,...,...,...,...
94,2025-09-04,0.83,NaN,1.89,0.28,NaN,NaN
95,2025-09-08,1.18,NaN,NaN,0.15,NaN,NaN
96,2025-09-16,1.26,19.11,0.42,0.49,NaN,NaN
97,2025-09-19,3.38,17.70,1.55,0.29,NaN,NaN


In [22]:


toxinas_mes_producto["Fecha Producción"] = pd.to_datetime(toxinas_mes_producto["Fecha Producción"])
COLORS = {
    "AFLATOXINA":  "#1C8074",  # verde principal
    "ZEARALENONA": "#1A494C",  # verde profundo
    "FUMONISINA":  "#94AF92",  # verde suave
    "DON":         "#666666",  # gris
}
GRID = "#FFFFFF"
BG_PAPER = "#FFFFFF"   # o prueba "#E6ECD8" si prefieres un fondo verdoso suave
BG_PLOT  = "#FFFFFF"

# Crear figura 2x2
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=["AFLATOXINA (ppb)", "ZEARALENONA (ppb)", "FUMONISINA (ppm)", "DON (ppm)"]
)

# Definir orden y unidades
cols = [
    ("AFLATOXINA", "ppb", 1, 1),
    ("ZEARALENONA", "ppb", 1, 2),
    ("FUMONISINA", "ppm", 2, 1),
    ("DON", "ppm", 2, 2)
]

# Trazas con colores corporativos
for col, unidad, r, c in cols:
    fig.add_trace(
        go.Scatter(
            x=toxinas_mes_producto["Fecha Producción"],
            y=toxinas_mes_producto[col],
            mode="lines+markers",
            name=col,
            line=dict(width=2, color=COLORS[col]),
            marker=dict(size=6, symbol="square")
        ),
        row=r, col=c
    )
    fig.update_yaxes(title_text=f"{col} ({unidad})", row=r, col=c)

# Estilo corporativo general
fig.update_layout(
    height=720, width=980,
    title="Evolución diaria de micotoxinas (2025+) en producto terminado",
    paper_bgcolor=BG_PAPER,
    plot_bgcolor=BG_PLOT,
    font=dict(family="Arial", size=12, color="#1A494C"),
    margin=dict(l=60, r=30, t=70, b=40),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Grillas suaves
fig.update_xaxes(showgrid=True, gridcolor=GRID, zeroline=False)
fig.update_yaxes(showgrid=True, gridcolor=GRID, zeroline=False)

# Guardar en HTML
#output_path = f"{ROOT_IMAGEN}/micotoxinas_subplots_evolucion.html"
fig.show()
s3.save_plotly_html(fig, "micotoxinas_subplots_evolucion")


In [26]:


toxinas_mes_producto["Fecha Producción"] = pd.to_datetime(toxinas_mes_producto["Fecha Producción"])
COLORS = {
    "AFLATOXINA":  "#1C8074",  # verde principal
    "ZEARALENONA": "#1A494C",  # verde profundo
    "FUMONISINA":  "#94AF92",  # verde suave
    "DON":         "#666666",  # gris
}
GRID = "#FFFFFF"
BG_PAPER = "#FFFFFF"   # o prueba "#E6ECD8" si prefieres un fondo verdoso suave
BG_PLOT  = "#FFFFFF"

# Crear figura 2x2
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=["AFLATOXINA (ppb)", "ZEARALENONA (ppb)", "FUMONISINA (ppm)", "DON (ppm)"]
)

# Definir orden y unidades
cols = [
    ("AFLATOXINA", "ppb", 1, 1),
    ("ZEARALENONA", "ppb", 1, 2),
    ("FUMONISINA", "ppm", 2, 1),
    ("DON", "ppm", 2, 2)
]

# Trazas con colores corporativos
for col, unidad, r, c in cols:
    fig.add_trace(
        go.Scatter(
            x=toxinas_mes_producto["Fecha Producción"],
            y=toxinas_mes_producto[col],
            mode="lines+markers",
            name=col,
            line=dict(width=2, color=COLORS[col]),
            marker=dict(size=6, symbol="square")
        ),
        row=r, col=c
    )
    fig.update_yaxes(title_text=f"{col} ({unidad})", row=r, col=c)

# Estilo corporativo general
fig.update_layout(
    height=720, width=980,
    title="Evolución diaria de micotoxinas (2025+) en producto terminado",
    paper_bgcolor=BG_PAPER,
    plot_bgcolor=BG_PLOT,
    font=dict(family="Arial", size=12, color="#1A494C"),
    margin=dict(l=60, r=30, t=70, b=40),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Grillas suaves
fig.update_xaxes(showgrid=True, gridcolor=GRID, zeroline=False)
fig.update_yaxes(showgrid=True, gridcolor=GRID, zeroline=False)

# Guardar en HTML
#output_path = f"{ROOT_IMAGEN}/micotoxinas_subplots_evolucion.html"
#fig.write_html(output_path, include_plotlyjs="cdn", full_html=True)

#print(f"Gráfico guardado en: {output_path}")
fig.show()
s3.save_plotly_html(fig, "micotoxinas_subplots_evolucion.html")

In [28]:


toxinas_mes_producto["Fecha Producción"] = pd.to_datetime(toxinas_mes_producto["Fecha Producción"])
COLORS = {
    "AFLATOXINA":  "#1C8074",  # verde principal
    "ZEARALENONA": "#1A494C",  # verde profundo
    "FUMONISINA":  "#94AF92",  # verde suave
    "DON":         "#666666",  # gris
}
GRID = "#FFFFFF"
BG_PAPER = "#FFFFFF"   # o prueba "#E6ECD8" si prefieres un fondo verdoso suave
BG_PLOT  = "#FFFFFF"

# Crear figura 2x2
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=["AFLATOXINA (ppb)", "ZEARALENONA (ppb)", "FUMONISINA (ppm)", "DON (ppm)"]
)

# Definir orden y unidades
cols = [
    ("AFLATOXINA", "ppb", 1, 1),
    ("ZEARALENONA", "ppb", 1, 2),
    ("FUMONISINA", "ppm", 2, 1),
    ("DON", "ppm", 2, 2)
]

# Trazas con colores corporativos
for col, unidad, r, c in cols:
    fig.add_trace(
        go.Scatter(
            x=toxinas_mes_producto["Fecha Producción"],
            y=toxinas_mes_producto[col],
            mode="lines+markers",
            name=col,
            line=dict(width=2, color=COLORS[col]),
            marker=dict(size=6, symbol="square")
        ),
        row=r, col=c
    )
    fig.update_yaxes(title_text=f"{col} ({unidad})", row=r, col=c)

# Estilo corporativo general
fig.update_layout(
    height=720, width=980,
    title="Evolución diaria de micotoxinas (2025+) en producto terminado",
    paper_bgcolor=BG_PAPER,
    plot_bgcolor=BG_PLOT,
    font=dict(family="Arial", size=12, color="#1A494C"),
    margin=dict(l=60, r=30, t=70, b=40),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Grillas suaves
fig.update_xaxes(showgrid=True, gridcolor=GRID, zeroline=False)
fig.update_yaxes(showgrid=True, gridcolor=GRID, zeroline=False)

# Guardar en HTML
#output_path = f"{ROOT_IMAGEN}/micotoxinas_subplots_evolucion.html"
#fig.write_html(output_path, include_plotlyjs="cdn", full_html=True)

#print(f"Gráfico guardado en: {output_path}")
fig.show()
s3.save_plotly_html(fig, "micotoxinas_subplots_evolucion.html")


In [29]:
toxinas["ESPECIE"].unique()

array(['GANADO LECH.', 'REPRODUCTORA', 'LEVANTE BROILER',
       'POSTURA PONEDORAS', 'CERDOS', 'FIN BROILER', 'CONEJO',
       'LEVANTE PONEDORAS', 'CAPRINO', 'GANADO ENG.', 'REPRODUCOTRA',
       'POSTURA PONEDORA', 'GANADO', nan, 'POSTURA', 'GANADO TERN.'],
      dtype=object)

In [30]:
toxinas["Fecha Producción"].max()

Timestamp('2025-09-22 00:00:00')

In [31]:
df_cor =  toxinas[toxinas["Fecha Producción"]>='2025-08-01']
df_cor["ESPECIE"].unique()

array(['POSTURA PONEDORAS', 'FIN BROILER', 'GANADO LECH.', 'GANADO ENG.'],
      dtype=object)

In [32]:
for esp in df_cor["ESPECIE"].unique():
    df_ = df_cor[df_cor["ESPECIE"] == esp]

    toxinas_mes_producto = df_.groupby(["Fecha Producción"]).agg(
        AFLATOXINA = ('AFLA-PPB', 'mean'),#
        #ocra = ('OCRA-PPB', 'mean'),
        #t2 = ('T2-PPB', 'mean'),
        ZEARALENONA = ('ZEARA-PPB', 'mean'),#
        FUMONISINA = ('FUMO-PPM', 'mean'),#
        DON = ('DON-PPM', 'mean'),#
        temp = ('temp', 'mean'),
        pct_hum = ('pct_hum', 'mean')

    ).reset_index()

    # Seleccionar solo las columnas numéricas de interés
    cols_corr = ["AFLATOXINA", "ZEARALENONA", "FUMONISINA", "DON"]

    # Matriz de correlación
    corr_matrix = toxinas_mes_producto[cols_corr].corr(method="kendall")  # puedes probar "spearman", "kendall"

    # Heatmap interactivo con Plotly
    fig = px.imshow(
        corr_matrix,
        text_auto=".2f",  # valores en cada celda
        aspect="auto",
        color_continuous_scale="RdBu_r",
        title=f"Matriz de correlación micotoxinas y variables -{ esp} -{df_.shape[0]}"
    )

    # Ajustes de layout
    fig.update_layout(
        width=800, height=600,
        font=dict(size=12, color="#1A494C"),
        margin=dict(l=60, r=30, t=70, b=40)
    )

    fig.show()


In [35]:

OKUO_DIVERGING = [
    [0.0,  "#1A494C"],  # negativo fuerte
    [0.5,  "#E6ECD8"],  # neutro
    [1.0,  "#1C8074"],  # positivo fuerte
]

for esp in df_cor["ESPECIE"].unique():
    df_ = df_cor[df_cor["ESPECIE"] == esp].copy()

    toxinas_mes_producto = (
        df_.groupby(["Fecha Producción"])
           .agg(
               AFLATOXINA=('AFLA-PPB', 'mean'),
               ZEARALENONA=('ZEARA-PPB', 'mean'),
               FUMONISINA=('FUMO-PPM', 'mean'),
               DON=('DON-PPM', 'mean'),
               temp=('temp', 'mean'),
               pct_hum=('pct_hum', 'mean'),
           )
           .reset_index()
    )

    # Solo micotoxinas (como pediste)
    cols_corr = ["AFLATOXINA", "ZEARALENONA", "FUMONISINA", "DON"]

    # Evita fallos si hay columnas vacías o todo NaN
    if toxinas_mes_producto[cols_corr].dropna(how="all").empty:
        print(f"[{esp}] No hay datos suficientes para correlación.")
        continue

    corr_matrix = toxinas_mes_producto[cols_corr].corr(method="kendall")

    # --- Dejar solo triángulo superior ---
    # máscara triángulo inferior (incluida la diagonal inferior)
    mask_lower = np.tril(np.ones_like(corr_matrix, dtype=bool), k=-1)
    corr_upper = corr_matrix.mask(mask_lower)  # NaN abajo

    # Heatmap con paleta corporativa y triángulo superior
    fig = px.imshow(
        corr_upper,
        text_auto=".2f",
        aspect="auto",
        color_continuous_scale=OKUO_DIVERGING,
        zmin=-1, zmax=1,  # fija límites simétricos
        origin="upper",
        title=f"Matriz de correlación para la especie: — {esp} —", # n={df_.shape[0]}
    )

    fig.update_traces(
        hovertemplate="X:%{y}<br>Y:%{x}<br>τ=%{z:.2f}<extra></extra>"
    )

    fig.update_layout(
        width=800, height=400,
        font=dict(size=12, color="#1A494C"),
        margin=dict(l=70, r=40, t=80, b=60),
        coloraxis_colorbar=dict(title="τ Kendall"),
        paper_bgcolor="#FFFFFF",
        plot_bgcolor="#FFFFFF",
    )

    # Opcional: angula etiquetas si son largas
    fig.update_xaxes(tickangle=30)
    fig.update_yaxes(autorange="reversed")  # para que diagonal quede arriba-izquierda
    fig.show()

    # Guardar HTML (y opcional PNG si tienes kaleido)
    s3.save_plotly_html(fig, f"correlacion_micotoxinas_{esp}.html")
    print(f"Guardado: correlacion_micotoxinas_{esp}.html")


Guardado: correlacion_micotoxinas_POSTURA PONEDORAS.html


Guardado: correlacion_micotoxinas_FIN BROILER.html


Guardado: correlacion_micotoxinas_GANADO LECH..html


Guardado: correlacion_micotoxinas_GANADO ENG..html
